# Frustum PointNets PyTorch Implementation

Pipeline
1. Backbone i.e. ResNet
2. FPN - Object Detection
3. Taking the Object Detection proposal in the 2D image and change them to 3D frustums.
4. Train the model for 3D frustums
- More details to follow

In [2]:
import torch
import torchvision
import torch.nn as nn



import matplotlib.pyplot

In [3]:
class PreActivationBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        
        super(PreActivationBlock, self).__init__()
        self.bn_1 = nn.BatchNorm2d(in_planes)
        self.conv_1 = nn.Conv2D(in_channels=in_planes, out_channels=out_planes, \
                                kernel_size=3, stride=stride, padding=1, \
                                bias = false)
        

In [5]:
class DetectionNet(nn.Module):
    def __init__(self):
        
        super(DetectionNet, self).__init__()
        
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3,padding=1)
        self.conv2_2 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3_1 = nn.Conv2d

In [7]:
model = torchvision.models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to C:\Users\Billi/.cache\torch\checkpoints\resnet101-5d3b4d8f.pth
100%|████████████████████████████████████████████████| 178728960/178728960 [00:15<00:00, 11361512.49it/s]


In [8]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F